# Extração dos textos das resoluções em pdf, gerando novo dataset

O objetivo é identificar e extrair o texto contido nas resoluções em PDF para cada protocolo identificado na lista consolidada do ProInfo (01 - ProInfo - Lista Consolidada v1_0.xlsx)

Os passos são:
- identificar se a página possui um cabeçalho (se possuir um número de protocolo válido);
- associar o número da página no arquivo do ProInfo;
- extrair o texto da página e inserir na tabela; e
- verificar se foram identificadas as páginas de resoluções para todos os protocolos do ProInfo.

Produtos:<br>
10 - Resoluções digitalizadas v1_0.xlsx<br>
 - PAG
 - PROTOCOLO
 - TEXTO_PDF
 - Relator

# 1 - Tratamentos iniciais
Criação de um Dataframe com informações sobre o arquivo PDF:
- identificação do Gabinete do relator // Esse passo pode ser eliminado
- identificação do protocolo do processo
- identificação inicial das páginas válidas
- As informações no DataFrame estarão relacionadas com o arquivo PDF pelo número da página

In [ ]:
# !pip install pdfplumber

In [1]:
import pandas as pd
import re
import pdfplumber

In [3]:
path_produtos = 'Dados\\Produtos\\'
path_originais = 'Dados\\Coletados\\'
path_ajustados = 'Dados\\Ajustados\\'

In [5]:
# Carrega DataSet gerado em "01 - Tratamento Inicial dos DataSets"
pdf = pdfplumber.open(path_ajustados+"Resoluções v1_0 ago-24.pdf")

In [6]:
# Função para extrair os textos dos PDF e estruturar o DataFrame com os conteúdos de um DataFrame por página do arquivo PDF identificando o Protocolo, Gabinete e conteúdo de texto.
def IndentificaPaginasValidas(resol_pdf):
    re_protocolo = re.compile(r'TCE/\d{4,6}/20\d\d') # Máscara de ER para identificação do número do protocolo
    re_gabinete = r'(Gabinete do Conselheiro|Gabinete da Conselheira|Gabinete Cons.):?\s?(.*?)(\W)?\n' # ER para identificar o Gabinete do Conselheiro
    df_cons = pd.DataFrame(columns=['PAG','GABINETE','PROTOCOLO', 'TEXTO_PDF'])
    for pagina in resol_pdf:
        bounding_box = (30,10,pagina.width-30, pagina.height-10) # Limites de página. Importante para evitar sujeira no texto causada por textos escritos na borda do PDF.
        pagina_pdf = pagina.crop(bounding_box, relative=False, strict=True)
        pagina_txt = pagina_pdf.extract_text(layout=False,   horizontal_ltr=True) #vertical_ttb=False,
        pagina_txt = re.sub(r'(TCE/) *(\d{4,6}) *(/20\d\d)', r'\1\2\3', pagina_txt) # Correção de espaços em branco no número do protocolo
        pagina_txt = re.sub(r'TCE/0*(\d{1,6})/(\d{4})', lambda x: f'TCE/{x.group(1).zfill(6)}/{x.group(2)}', pagina_txt) # Correção da quantidade de zeros a esquerda no protocolo.
        protocolo_search = re_protocolo.search(pagina_txt)  # Identifica se tem e qual o número do protocolo
        if protocolo_search:
            protocolo = protocolo_search[0]     
            gabinete_search = re.search(re_gabinete, pagina_txt, re.DOTALL|re.IGNORECASE|re.ASCII) # Identifica qual o Gabinete
            if gabinete_search:
                gabinete = gabinete_search.group(2)
            else:
                gabinete = pd.NA
        else:
            protocolo = pd.NA
            gabinete = pd.NA
        nova_linha = {'PAG':[pagina.page_number], 'PROTOCOLO':[protocolo],'GABINETE':[gabinete], 'TEXTO_PDF':[pagina_txt]}
        df_cons = pd.concat([df_cons, pd.DataFrame(nova_linha)])
        
    return df_cons


In [7]:
# Executa a função para extrair os textos dos PDF e estruturar o DataFrame com os conteúdos
# e salva o resultado em disco
# "ESSE CÓDIGO DEMORA PARA SER EXECUTADO (7min)"

df_resol_ini = IndentificaPaginasValidas(pdf.pages)


ModuleNotFoundError: No module named 'openpyxl'

In [8]:
import openpyxl
df_resol_ini.to_excel(path_ajustados+'10 - Resoluções Extraídas v1_0.xlsx')

# Verifica se todos os protocolos do ProInfo (df_proinfo) foram carregados no arquivo de resoluções (df_resol_ini)

In [ ]:
# Carrega nas variáveis o resultado da extração da função IndentificaPaginasValidas()
df_resol_ini = pd.read_excel(path_ajustados+'10 - Resoluções Extraídas v1_0.xlsx', index_col=0)
df_resol_ini.reset_index(drop=True, inplace=True)
df_proinfo = pd.read_excel(path_produtos+'01 - ProInfo - Lista Consolidada v1_0.xlsx', usecols=['Número','Ano','Protocolo','Natureza','Relator','Exercício'])

In [ ]:
df_resol_ini.info()

In [ ]:
df_proinfo

In [ ]:
set_resol = set(df_resol_ini['PROTOCOLO'])
set_proinfo = set(df_proinfo['Protocolo'])
print(f'Número de protocolos em Resoluções = {len(set_resol)}')
print(f'Número de protocolos em ProInfo = {len(set_proinfo)}')
print(f'Quantidade de protocolos existentes em Resoluções faltando em ProInfo = {len(set_resol - set_proinfo)}')
print(f'Quantidade de protocolos existentes em ProInfo faltando em Resoluções = {len(set_proinfo - set_resol)}')
investigar = set_proinfo - set_resol # Protocolos do ProInfo não identificado em Resoluções


In [ ]:
investigar

In [ ]:
# Nova rodada para identificação de páginas com protocolos e extração dos textos dos PDF além de estruturar o DataFrame com os conteúdos de um DataFrame por página do arquivo PDF identificando o Protocolo, Gabinete e conteúdo de texto.

def IndentificaPaginasValidas2(resol_pdf):
    re_protocolo = re.compile(r'TCE\s?[/\\]? ?\d{3,9}/20\d\d') # Máscara de ER para identificação do número do protocolo
    re_gabinete = r'(Gabinete do Conselheiro|Gabinete da Conselheira|Gabinete Cons.):?\s?(.*?)(\W)?\n' # ER para identificar o Gabinete do Conselheiro
    df_cons = pd.DataFrame(columns=['PAG','GABINETE','PROTOCOLO', 'TEXTO_PDF'])
    ind=0
    for pagina in resol_pdf:
        print(ind)
        if pd.isna(df_resol_ini.loc[ind, 'PROTOCOLO']): # Executa a rotina para as páginas que não foram encontradas o protocolo na primeira avaliação
            bounding_box = (30,10,pagina.width-30, pagina.height-10) # Limites de página. Importante para evitar sujeira no texto causada por textos escritos na borda do PDF.
            pagina_pdf = pagina.crop(bounding_box, relative=False, strict=True)
            pagina_txt = pagina_pdf.extract_text(layout=False,   horizontal_ltr=True) #vertical_ttb=False,
            pagina_txt = re.sub(r'(TCE/) *(\d{4,6}) *(/20\d\d)', r'\1\2\3', pagina_txt) # Correção de espaços em branco no número do protocolo
            pagina_txt = re.sub(r'TCE/0*(\d{1,6})/(\d{4})', lambda x: f'TCE/{x.group(1).zfill(6)}/{x.group(2)}', pagina_txt) # Correção da quantidade de zeros a esquerda no protocolo.
            protocolo_search = re_protocolo.search(pagina_txt)  # Identifica se tem e qual o número do protocolo
            if protocolo_search:
                protocolo = protocolo_search[0]     
                gabinete_search = re.search(re_gabinete, pagina_txt, re.DOTALL|re.IGNORECASE|re.ASCII) # Identifica qual o Gabinete
                if gabinete_search:
                    gabinete = gabinete_search.group(2)
                else:
                    gabinete = pd.NA
            else:
                protocolo = pd.NA
                gabinete = pd.NA
            nova_linha = {'PAG':[pagina.page_number], 'PROTOCOLO':[protocolo],'GABINETE':[gabinete], 'TEXTO_PDF':[pagina_txt]}
            df_cons = pd.concat([df_cons, pd.DataFrame(nova_linha)])
            ind=ind+1
        else:
            ind=ind+1
            continue

    return df_cons

df_resol_ini2 = IndentificaPaginasValidas2(pdf.pages)


In [ ]:
df_resol_ini2.loc[df_resol_ini2['PROTOCOLO'] == 'TCE003416/2013', 'PROTOCOLO'] = 'TCE/003416/2013'

In [ ]:
protocolos_ajustados = []

In [ ]:
df_resol_ini2['PROTOCOLO'].value_counts()

In [ ]:
df_resol_ini2['PROTOCOLO'] = df_resol_ini2['PROTOCOLO'].apply(lambda valor: valor.replace('TCE /', 'TCE/').replace('TCE 0', 'TCE/0') if pd.notna(valor) else valor)
df_resol_ini2['PROTOCOLO'].value_counts()

In [ ]:
# Substituir as linhas em df_resol com base na coluna "PAG"
df_resol_ini.set_index('PAG', inplace=True)
df_resol_ini2.set_index('PAG', inplace=True)
#df_resol_ini2.set_index('PAG', inplace=True)

df_resol_ini.update(df_resol_ini2)

# Resetar o índice, se necessário
df_resol_ini.reset_index(inplace=True)



In [ ]:
set_resol = set(df_resol_ini['PROTOCOLO'])
set_proinfo = set(df_proinfo['Protocolo'])
print(f'Número de protocolos em Resoluções = {len(set_resol)}')
print(f'Número de protocolos em ProInfo = {len(set_proinfo)}')
print(f'Quantidade de protocolos existentes em Resoluções faltando em ProInfo = {len(set_resol - set_proinfo)}')
print(f'Quantidade de protocolos existentes em ProInfo faltando em Resoluções = {len(set_proinfo - set_resol)}')
investigar = set_proinfo - set_resol

In [ ]:
investigar

In [ ]:
df_resol_ini.info()

In [ ]:
lista = []
for valor in df_resol_ini['PROTOCOLO']:
    if pd.isna(valor):
        lista.append(0)
    else:
        lista.append(len(valor))

In [ ]:
df_resol_ini['lista'] = lista
df_resol_ini.loc[(df_resol_ini['lista'] ==13)|(df_resol_ini['lista'] ==14)]

In [ ]:
for ind in df_resol_ini.loc[df_resol_ini['lista']==13, 'PROTOCOLO'].index:
    df_resol_ini.loc[ind, 'PROTOCOLO'] = df_resol_ini.loc[ind, 'PROTOCOLO'].replace('TCE/', 'TCE/00')

for ind in df_resol_ini.loc[df_resol_ini['lista']==14, 'PROTOCOLO'].index:
    print(df_resol_ini.loc[ind, 'PROTOCOLO'].replace('TCE/', 'TCE/0'))
    df_resol_ini.loc[ind, 'PROTOCOLO'] = df_resol_ini.loc[ind, 'PROTOCOLO'].replace('TCE/', 'TCE/0')

In [ ]:
df_resol_ini.loc[(df_resol_ini['lista'] ==13)|(df_resol_ini['lista'] ==14)]

In [ ]:
set_resol = set(df_resol_ini['PROTOCOLO'])
set_proinfo = set(df_proinfo['Protocolo'])
print(f'Número de protocolos em Resoluções = {len(set_resol)}')
print(f'Número de protocolos em ProInfo = {len(set_proinfo)}')
print(f'Quantidade de protocolos existentes em Resoluções faltando em ProInfo = {len(set_resol - set_proinfo)}')
print(f'Quantidade de protocolos existentes em ProInfo faltando em Resoluções = {len(set_proinfo - set_resol)}')
investigar = set_proinfo - set_resol

In [ ]:
df_proinfo.loc[df_proinfo['Protocolo'].isin(investigar), ['Número', 'Ano', 'Protocolo']]

# Análise a partir do número da resolução<br>
 223/2013	TCE/000502/2012:4762<br>
 253/2013	TCE/010805/2002:4818<br>
 258/2013	TCE/002978/2008:4800<br>
 238/2014	TCE/001234/2007:5073<br>
 560/2014	TCE/000413/2013:5365<br>
 655/2014	TCE/005132/2003:5453<br>
 656/2014	TCE/000781/2006:5454<br>
 8/2015		TCE/002936/2006:1915<br>
 111/2015	TCE/000079/2010:2003<br>
 253/2015	TCE/001134/2010:2119<br>
 287/2015	TCE/003532/2013:2145<br>
 407/2015	TCE/002493/2004:2297<br>
 421/2015	TCE/005953/2003:1726<br>
 72/2017	TCE/004178/2008:2817<br>
 127/2017	TCE/005510/2016:2878<br>
 114/2018	TCE/001895/2007:3053<br>
 132/2018	TCE/003429/2011:3078<br>
 6/2020		TCE/011678/2019:3442<br>
 119/2022	TCE/008021/2019:1041<br>
 162/2023	TCE/006338/2021: desentranhada

In [ ]:
 223/2013	TCE/000502/2012:68
 253/2013	TCE/010805/2002:124
 258/2013	TCE/002978/2008:106
 238/2014	TCE/001234/2007:379
 560/2014	TCE/000413/2013:671
 655/2014	TCE/005132/2003:759
 656/2014	TCE/000781/2006:760
 8/2015		TCE/002936/2006:970
 111/2015	TCE/000079/2010:1067
 253/2015	TCE/001134/2010:1198
 287/2015	TCE/003532/2013:1227
 407/2015	TCE/002493/2004:1391
 421/2015	TCE/005953/2003:1398
 72/2017	TCE/004178/2008:2014
 127/2017	TCE/005510/2016:2088
 114/2018	TCE/001895/2007:2283
 132/2018	TCE/003429/2011:2312
 6/2020		TCE/011678/2019:2709
 119/2022	TCE/008021/2019:5261
 162/2023	TCE/006338/2021: desentranhada

In [ ]:
ajustes = {'Protocolo':['TCE/000502/2012','TCE/010805/2002','TCE/002978/2008','TCE/001234/2007','TCE/000413/2013','TCE/005132/2003','TCE/000781/2006','TCE/002936/2006','TCE/000079/2010','TCE/001134/2010','TCE/003532/2013','TCE/002493/2004','TCE/005953/2003','TCE/004178/2008','TCE/005510/2016','TCE/001895/2007','TCE/003429/2011','TCE/011678/2019','TCE/008021/2019'],
        'PAG':[68, 124, 106, 379, 671, 759, 760, 970, 1067, 1198, 1227, 1391, 1398, 2014, 2088, 2283, 2312, 2709, 5261]}
df_ajustes = pd.DataFrame(ajustes)
df_ajustes

In [ ]:
for ind in df_ajustes.index:
    df_resol_ini.loc[df_resol_ini['PAG']==df_ajustes.loc[ind, 'PAG'], 'PROTOCOLO'] = df_ajustes.loc[ind, 'Protocolo']

In [ ]:
df_resol_ini[df_resol_ini['PROTOCOLO']=='TCE/000079/2010']

In [ ]:
set_resol = set(df_resol_ini['PROTOCOLO'])
set_proinfo = set(df_proinfo['Protocolo'])
print(f'Número de protocolos em Resoluções = {len(set_resol)}')
print(f'Número de protocolos em ProInfo = {len(set_proinfo)}')
print(f'Quantidade de protocolos existentes em Resoluções faltando em ProInfo = {len(set_resol - set_proinfo)}')
print(f'Quantidade de protocolos existentes em ProInfo faltando em Resoluções = {len(set_proinfo - set_resol)}')
investigar = set_proinfo - set_resol

In [ ]:
df_proinfo = df_proinfo[~df_proinfo['Protocolo'].isin(investigar)]

In [ ]:
set_resol = set(df_resol_ini['PROTOCOLO'])
set_proinfo = set(df_proinfo['Protocolo'])
print(f'Número de protocolos em Resoluções = {len(set_resol)}')
print(f'Número de protocolos em ProInfo = {len(set_proinfo)}')
print(f'Quantidade de protocolos existentes em Resoluções faltando em ProInfo = {len(set_resol - set_proinfo)}')
print(f'Quantidade de protocolos existentes em ProInfo faltando em Resoluções = {len(set_proinfo - set_resol)}')
investigar = set_proinfo - set_resol

In [ ]:
# Gera o arquivo final
set_resol = set(df_resol_ini['PROTOCOLO'])
set_proinfo = set(df_proinfo['Protocolo'])
df_resol = df_resol_ini[df_resol_ini['PROTOCOLO'].isin(set_proinfo.intersection(set_resol))]

In [ ]:
set_resol = set(df_resol['PROTOCOLO'])
set_proinfo = set(df_proinfo['Protocolo'])
print(f'Número de protocolos em Resoluções = {len(set_resol)}')
print(f'Número de protocolos em ProInfo = {len(set_proinfo)}')
print(f'Quantidade de protocolos existentes em Resoluções faltando em ProInfo = {len(set_resol - set_proinfo)}')
print(f'Quantidade de protocolos existentes em ProInfo faltando em Resoluções = {len(set_proinfo - set_resol)}')
investigar = set_proinfo - set_resol

In [ ]:
df_resol.drop('lista', axis=1, inplace=True)
df_resol.to_excel(path_ajustados+'10 - Resoluções Extraídas v2_0.xlsx')

# Juntando os textos extraídos do PDF com os dados extraídos do ProInfo,

In [3]:
df_resol = pd.read_excel(path_ajustados+'10 - Resoluções Extraídas v2_0.xlsx', usecols=['PAG','PROTOCOLO', 'TEXTO_PDF'])
df_proinfo = pd.read_excel(path_produtos+'01 - ProInfo - Lista Consolidada v1_0.xlsx', usecols=['Protocolo','Relator'])

In [4]:
df_proinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525 entries, 0 to 2524
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Protocolo  2525 non-null   object
 1   Relator    2525 non-null   object
dtypes: object(2)
memory usage: 39.6+ KB


In [5]:
df_resol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2728 entries, 0 to 2727
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   PAG        2728 non-null   int64 
 1   PROTOCOLO  2728 non-null   object
 2   TEXTO_PDF  2728 non-null   object
dtypes: int64(1), object(2)
memory usage: 64.1+ KB


In [6]:
# Verifica situações em que o texto de uma resolução possui mais de uma página e o número do protocolo foi identificado em várias páginas. A página válida é a primeira.
# Essa situação é diferente de quando um processo possui recurso julgado posteriormente. Nesse caso o protocolo é igual, mas as páginas não são sequenciais e a válida é a mais nova.


def RemoveProtocolosComVariasPáginas(df_ref):
    # Filtra os registros com protocolos duplicados
    temp = df_ref[df_ref['PROTOCOLO'].duplicated(keep=False)].sort_values(['PROTOCOLO', 'PAG'])
    
    # Filtra os protocolos duplicados em páginas sequenciais.
    filtered_temp = temp[temp.apply(lambda row: (row['PAG'] + 1) in temp[(temp['PROTOCOLO'] == row['PROTOCOLO'])]['PAG'].values or
                                    (row['PAG'] - 1) in temp[(temp['PROTOCOLO'] == row['PROTOCOLO'])]['PAG'].values, axis=1)] 

    # Remove os registros duplicados em df_resol, mantendo apenas o primeiro por protocolo duplicado
    duplicated_protocols = filtered_temp['PROTOCOLO'].unique()

    df_resol_updated = pd.concat([
        df_ref[~df_ref['PROTOCOLO'].isin(duplicated_protocols)],
        filtered_temp.drop_duplicates(subset=['PROTOCOLO'], keep='first')
    ]).sort_values(['PROTOCOLO', 'PAG'])

    return df_resol_updated

df_resol = RemoveProtocolosComVariasPáginas(df_resol)
df_resol.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2678 entries, 2111 to 421
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   PAG        2678 non-null   int64 
 1   PROTOCOLO  2678 non-null   object
 2   TEXTO_PDF  2678 non-null   object
dtypes: int64(1), object(2)
memory usage: 83.7+ KB


In [7]:
# Elimina os protocolos duplicados em resoluções diferentes, mantendo o mais recente.

df_resol.drop_duplicates(subset=['PROTOCOLO'], keep='last', inplace=True) # Apagas os protocolos que foram alterados posteriormente. Mantém o último.
df_resol.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2524 entries, 2111 to 421
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   PAG        2524 non-null   int64 
 1   PROTOCOLO  2524 non-null   object
 2   TEXTO_PDF  2524 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.9+ KB


In [8]:
df_proinfo.rename(columns={'Protocolo': 'PROTOCOLO'}, inplace=True)

In [9]:
df_resol2 = df_resol.merge(df_proinfo, on='PROTOCOLO', how='left')

In [10]:
df_proinfo2 = df_proinfo.merge(df_resol, on='PROTOCOLO', how='left')

In [11]:
df_proinfo2[df_proinfo2['TEXTO_PDF'].isna()]

,PROTOCOLO,Relator,PAG,TEXTO_PDF
2300,TCE/006338/2021,Carolina Matos,NaN,NaN


In [12]:
df_resol2.loc[df_resol2['Relator'] == 'JUSTINIANO ZILTON ROCHA', 'Relator'] = 'Justiniano Zilton Rocha'

In [13]:
df_resol2['Relator'].value_counts()

Relator
Pedro Henrique Lino de Souza                     744
Joao Evilasio Vasconcelos Bonfim                 399
Gildasio Penedo Filho                            316
Inaldo da Paixão Santos Araújo                   265
Marcus Vinícius de Barros Presídio               244
Carolina Matos                                   210
Antonio Honorato de Castro Neto                  208
José Eduardo Vieira Zezéu Ribeiro                 82
Justiniano Zilton Rocha                           25
(Na Vacância) Almir Pereira da Silva              15
(Na Vacância) Maria do Carmo Galvao do Amaral     12
Antônio França Teixeira                            2
(Na Vacância) Sergio Spector                       2
Name: count, dtype: int64

In [15]:
df_resol2.rename(columns={'Relator': 'RELATOR'}, inplace=True)

In [16]:
df_resol2.to_excel(path_produtos+'10 - Resoluções digitalizadas v1_0.xlsx')